## Carga Datajud



## Imports






In [12]:
import json
from datetime import datetime
from google.colab import auth
from googleapiclient.discovery import build
from google.cloud import storage
import csv
from io import StringIO 
import traceback
import logging


## Autentica usuário no Google Colab

In [13]:
auth.authenticate_user()


## Configura acesso ao Google Storage

In [5]:
BUCKET_NAME = "ueizejud"
project='tensile-virtue-291314'

gcs_service = build('storage', 'v1')
client = storage.Client(project=project)
bucket = storage.Bucket(client, name=BUCKET_NAME, user_project=project)

## Parser para arquivos json

In [10]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
#logger.setLevel(logging.DEBUG)


def obtem_valor(dicionario, chave, padrao=''):
  if chave in dicionario and dicionario[chave] != None:
    return dicionario[chave]
  else:
    return padrao


campos = {
'processos':
       [
			'id_processo', # see  gera_id_processo (proc)  #'id_processo'  TRT9_1125_G1_20393_00000014220195090325.json
			'millisInsercao', # millisInsercao: campo interno com data e hora em milissegundos que representa quando o arquivo foi indexado no Datajud;			
			'grau', # grau: Campo identificador do grau de jurisdição em que o processo se encontra; valores possíveis: - 'SUP': Tribunal Superior - 'G2': 2º Grau - 'TR': Turma Recursal - 'G1': 1º grau, justiça comum - 'JE': Juizados Especiais (inclusive os regidos pelas leis dos Juizados Especiais 9.099/95 e 12.153/2009 que tramitam em varas de juízo único ou em varas que acumulam competência de juizado especial) - 'TRU': Turma Regional de Uniformização - 'TNU': Turma Nacional de Uniformização - 'TEU': Turma Estadual de Uniformização - 'CJF': Conselho da Justiça Federal - 'CSJT': Conselho Superior da Justiça do Trabalho;
			'siglaTribunal', #siglaTribunal: campo identificador da sigla do Tribunal em que o processo se encontra; 			
			'processoVinculado', #dadosBasicos.processoVinculado: elemento destinado a permitir a indicação da existência de um ou mais processos judiciais vinculados.
			'relacaoIncidental', #dadosBasicos.relacaoIncidental: elemento destinado a permitir identificar se existe algum elemento incidental que tenha gerado novo processo judicial, em razão da existência de uma nova relação processual jurídica (por exemplo, como pode ocorrer em embargos à execução de títulos extrajudiciais, embargos de terceiro, em recursos internos, impugnações ao valor da causa, entre outras Situações, em que há criação de novos autos vinculados ao processo principal);
			'prioridade', #dadosBasicos.prioridade: Elemento destinado a permitir a identificação da existência de propriedades processuais não óbvias, ou seja, aquelas que não são resultado direto da identificação da classe processual (ex.: habeas corpus ou mandado de segurança). Na versão 2.0, será texto livre, mas é recomendável utilizar os seguintes textos identificadores: "IDOSO" "RÉU PRESO" "PERECIMENTO" "MENOR";
			'valorCausa', #dadosBasicos.valorCausa: valor da causa;
			'orgaoJulgador_codigoOrgao', #dadosBasicos.orgaoJulgador.codigoOrgao: código da Unidade Judiciária constante no Módulo de Produtividade, Anexo II da Resolução 76;
			'orgaoJulgador_codigoMunicipioIBGE', #dadosBasicos.orgaoJulgador.codigoMunicipioIBGE: município-sede da unidade judiciária, conforme código de municípios definidos pelo IBGE. Usar código com sete dígitos. Fonte: www.ibge.gov.br;
      'orgaoJulgador_instancia', #dadosBasicos.orgaoJulgador.instancia: os tipos de instância podem ser: - ORIG: instância originária em que o processo teve início; - REV: instância de revisão direta de um processo originariamente proposto em outra instância; - ESP: instância de revisão especial de processo submetido ou não à revisão direta; - EXT: instância de revisão extraordinária - ADM: instância administrativa de análise;
      'orgaoJulgador_nomeOrgao', #dadosBasicos.orgaoJulgador.nomeOrgao: descrição textual da Unidade Judiciária constante no Módulo de Produtividade, Anexo II da Resolução 76;      
			'outrosnumeros', #dadosBasicos.outrosnumeros: outros números que o processo possa ter recebido durante sua vida;
			'numero', #dadosBasicos.numero: numeração única do processo conforme determinado pela Resolução 65;
			'competencia', #dadosBasicos.competencia: identificador da competência a que pertence o processo, ou da competência a que ele se destina caso se trate de processo inicial;
			'classeProcessual', #dadosBasicos.classeProcessual: código da classe processual conforme Resolução 46. Relação com sgt_classes.csv;
			'codigoLocalidade', #			dadosBasicos.codigoLocalidade: código identificador da localidade a que pertence ou deve pertencer o processo. Deve ser utilizado o código do município, segundo códigos do IBGE disponíveis em www.ibge.gov.br (usar códigos com sete dígitos);
			'nivelSigilo', #dadosBasicos.nivelSigilo: nível de sigilo a ser aplicado ao processo;
			'intervencaoMP', #dadosBasicos.intervencaoMP: atributo destinado a identificar que o processo exige a intervenção do Ministério Público;
			'tamanhoProcesso', #dadosBasicos.tamanhoProcesso: volume, em bytes, dos documentos existentes no processo judicial; extraordinária, deve refletir a data de entrada do processo nessa instância;
      'dataAjuizamento', #ndica a data em que o processo foi inicialmente recebido pelo Poder Judiciário no órgão consultado. Caso se trate de instância recursal, especial ou extraordinária, deve refletir a data de entrada do processo nessa instância;
      'procEl', #dadosBasicos.procEl: campo identifica se o processo tramita em sistema eletrônico ou em papel. São valores possíveis - 1: Sistema Eletrônico - 2: Sistema Físico;
			'dscSistema' #dadosBasicos.dscSistema: identifica qual o sistema eletrônico que o processo tramita. São valores possíveis: 1 – Pje, 2 – Projudi, 3 – SAJ, 4 – EPROC, 5 – Apolo, 6 – Themis, 7 – Libra, 8 – Outros;            			
			],  
'assuntos_processo': 
      [
      'id_processo',
      'codigoNacional', #dadosBasicos.assunto.codigoNacional: elemento destinado a que se informe o código de assunto existente na tabela nacional unificada decorrente da Resolução 46;
      'principal', #dadosBasicos.assunto.principal: atributo do tipo boolean que informa se o assunto referido é o assunto principal do processo.
      'assuntoLocal_codigoAssunto', #dadosBasicos.assunto.assuntoLocal.codigoAssunto: atributo destinado a incluir a informação relativa ao código numérico utilizado localmente pelo tribunal.
			'assuntoLocal_codigoPaiNacional', #dadosBasicos.assunto.assuntoLocal.codigoPaiNacional: atributo destinado à entrada do código de assunto nacional de que o assunto local é filho.
			'assuntoLocal_descricao' #dadosBasicos.assunto.assuntoLocal.descricao: atributo destinado à entrada da descrição textual do assunto local.
      ],
'movimentos_processo':
      [
		  'id_processo', #?
		  'dataHora', ##movimento.dataHora: atributo destinado a indicar o momento em que foi realizada a movimentação.
			'nivelSigilo', #movimento.nivelSigilo: nível de sigilo a ser aplicado ao processo;
			'identificadorMovimento', #movimento.identificadorMovimento: atributo incluído para permitir a atribuição de um identificador específico para a movimentação realizada em um determinado processo judicial.
			'tipoResponsavelMovimento', #movimento.tipoResponsavelMovimento: identificação do responsável pelo movimento: Servidor=0; Magistrado=1;
			'idDocumentoVinculado', #movimento.idDocumentoVinculado: elemento destinado a permitir a vinculação de um ou mais documentos à movimentação;
			'orgaoJulgador_codigoMunicipioIBGE', #movimento.orgaoJulgador.dadosBasicos.codigoMunicipioIBGE: município-sede da unidade judiciária, conforme código de municípios definidos pelo IBGE. Usar código com sete dígitos. Fonte: www.ibge.gov.br;
			'orgaoJulgador_codigoOrgao', #movimento.orgaoJulgador.dadosBasicos.codigoOrgao: código da Unidade Judiciária constante no Módulo de Produtividade, Anexo II da Resolução 76. 
			'orgaoJulgador_instancia', #movimento.orgaoJulgador.dadosBasicos.instancia: os tipos de instância podem ser: - ORIG: instância originária em que o processo teve início; - REV: instância de revisão direta de um processo originariamente proposto em outra instância; - ESP: instância de revisão especial de processo submetido ou não à revisão direta; - EXT: instância de revisão extraordinária - ADM: instância administrativa de análise;
			'tipoDecisao', #movimento.tipoDecisao: atributo que permite a atribuição da decisão como sendomonocrática (proferida por um magistrado), ou colegiada;Valores possíveis são numéricos (0 ou 1):0 - decisão MONOCRATICA1 - decisão COLEGIADA;
			'movimentoNacional_codigoNacional', #movimento.movimentoNacional.codigoNacional: atributo destinado à indicação do código do movimento previsto na tabela unificada de que trata a Resolução 46;
			'movimentoLocal.codigoMovimento', #movimento.movimentoLocal.codigoMovimento: atributo destinado a incluir a informação relativa ao código numérico utilizado localmente pelo tribunal;
			'movimentoLocal_codigoPaiNacional' #movimento.movimentoLocal.codigoPaiNacional: atributo destinado à entrada do código de movimento nacional de que o movimento local é filho;
          ],
'complementos_movimento':
			[
			'identificadorMovimento', #identificador movimento / chave
			'complementoNacional_codComplemento', #	movimento.complementoNacional.codComplemento: atributo destinado à entrada do código docomplemento do movimentonacional ou do movimento local;
			'complementoNacional_descricaoComplemento', #	movimento.complementoNacional.descricaoComplemento: atributo destinado à entrada da descrição textualdocomplemento do movimento nacional ou do movimento local;
			'movimento_codComplementoTabelado' #movimento.complementoNacional.codComplementoTabelado: atributo destinado à entrada do código do complemento tabelado do movimento nacional ou do movimento local;
			]
}

def gera_id_processo (proc):
  #'id_processo'  TRT9_1125_G1_20393_00000014220195090325.json
  return str(proc['siglaTribunal'])+'_'+str(proc['dadosBasicos']['classeProcessual'])+'_'+str(proc['grau'])+'_'+str(proc['dadosBasicos']['orgaoJulgador']['codigoOrgao'])+'_'+str(proc['dadosBasicos']['numero'])  

def d_proc(proc):
  campos_proc = campos['processos']
  yield [
      gera_id_processo (proc), #'id_processo'  TRT9_1125_G1_20393_00000014220195090325.json
      proc['millisInsercao'], # millisInsercao: campo interno com data e hora em milissegundos que representa quando o arquivo foi indexado no Datajud;			
			proc['grau'], # grau: Campo identificador do grau de jurisdição em que o processo se encontra; valores possíveis: - 'SUP': Tribunal Superior - 'G2': 2º Grau - 'TR': Turma Recursal - 'G1': 1º grau, justiça comum - 'JE': Juizados Especiais (inclusive os regidos pelas leis dos Juizados Especiais 9.099/95 e 12.153/2009 que tramitam em varas de juízo único ou em varas que acumulam competência de juizado especial) - 'TRU': Turma Regional de Uniformização - 'TNU': Turma Nacional de Uniformização - 'TEU': Turma Estadual de Uniformização - 'CJF': Conselho da Justiça Federal - 'CSJT': Conselho Superior da Justiça do Trabalho;
			proc['siglaTribunal'], #siglaTribunal: campo identificador da sigla do Tribunal em que o processo se encontra; 			
			obtem_valor(proc['dadosBasicos'], 'processoVinculado', padrao=''), #dadosBasicos.processoVinculado: elemento destinado a permitir a indicação da existência de um ou mais processos judiciais vinculados.
			obtem_valor(proc['dadosBasicos'], 'relacaoIncidental', padrao=''), #dadosBasicos.relacaoIncidental: elemento destinado a permitir identificar se existe algum elemento incidental que tenha gerado novo processo judicial, em razão da existência de uma nova relação processual jurídica (por exemplo, como pode ocorrer em embargos à execução de títulos extrajudiciais, embargos de terceiro, em recursos internos, impugnações ao valor da causa, entre outras Situações, em que há criação de novos autos vinculados ao processo principal);
			obtem_valor(proc['dadosBasicos'], 'prioridade', padrao=''), #dadosBasicos.prioridade: Elemento destinado a permitir a identificação da existência de propriedades processuais não óbvias, ou seja, aquelas que não são resultado direto da identificação da classe processual (ex.: habeas corpus ou mandado de segurança). Na versão 2.0, será texto livre, mas é recomendável utilizar os seguintes textos identificadores: "IDOSO" "RÉU PRESO" "PERECIMENTO" "MENOR";
			obtem_valor(proc['dadosBasicos'], 'valorCausa', padrao=''), #dadosBasicos.valorCausa: valor da causa;
			obtem_valor(proc['dadosBasicos']['orgaoJulgador'], 'codigoOrgao', padrao=''), #dadosBasicos.orgaoJulgador.codigoOrgao: código da Unidade Judiciária constante no Módulo de Produtividade, Anexo II da Resolução 76;
			obtem_valor(proc['dadosBasicos']['orgaoJulgador'], 'codigoMunicipioIBGE', padrao=''), #dadosBasicos.orgaoJulgador.codigoMunicipioIBGE: município-sede da unidade judiciária, conforme código de municípios definidos pelo IBGE. Usar código com sete dígitos. Fonte: www.ibge.gov.br;
      obtem_valor(proc['dadosBasicos']['orgaoJulgador'], 'instancia', padrao=''), #dadosBasicos.orgaoJulgador.instancia: os tipos de instância podem ser: - ORIG: instância originária em que o processo teve início; - REV: instância de revisão direta de um processo originariamente proposto em outra instância; - ESP: instância de revisão especial de processo submetido ou não à revisão direta; - EXT: instância de revisão extraordinária - ADM: instância administrativa de análise;
      obtem_valor(proc['dadosBasicos']['orgaoJulgador'], 'nomeOrgao', padrao=''), #dadosBasicos.orgaoJulgador.nomeOrgao: descrição textual da Unidade Judiciária constante no Módulo de Produtividade, Anexo II da Resolução 76;      
			obtem_valor(proc['dadosBasicos'], 'outrosnumeros', padrao=''), #dadosBasicos.outrosnumeros: outros números que o processo possa ter recebido durante sua vida;
			obtem_valor(proc['dadosBasicos'], 'numero', padrao=''), #dadosBasicos.numero: numeração única do processo conforme determinado pela Resolução 65;
			obtem_valor(proc['dadosBasicos'], 'competencia', padrao=''), #dadosBasicos.competencia: identificador da competência a que pertence o processo, ou da competência a que ele se destina caso se trate de processo inicial;
			obtem_valor(proc['dadosBasicos'], 'classeProcessual', padrao=''), #dadosBasicos.classeProcessual: código da classe processual conforme Resolução 46. Relação com sgt_classes.csv;
			obtem_valor(proc['dadosBasicos'], 'codigoLocalidade', padrao=''), #			dadosBasicos.codigoLocalidade: código identificador da localidade a que pertence ou deve pertencer o processo. Deve ser utilizado o código do município, segundo códigos do IBGE disponíveis em www.ibge.gov.br (usar códigos com sete dígitos);
			obtem_valor(proc['dadosBasicos'], 'nivelSigilo', padrao=''), #dadosBasicos.nivelSigilo: nível de sigilo a ser aplicado ao processo;
			obtem_valor(proc['dadosBasicos'], 'intervencaoMP', padrao=''), #dadosBasicos.intervencaoMP: atributo destinado a identificar que o processo exige a intervenção do Ministério Público;
			obtem_valor(proc['dadosBasicos'], 'tamanhoProcesso', padrao=''), #dadosBasicos.tamanhoProcesso: volume, em bytes, dos documentos existentes no processo judicial; extraordinária, deve refletir a data de entrada do processo nessa instância;
      obtem_valor(proc['dadosBasicos'], 'dataAjuizamento', padrao=''), #ndica a data em que o processo foi inicialmente recebido pelo Poder Judiciário no órgão consultado. Caso se trate de instância recursal, especial ou extraordinária, deve refletir a data de entrada do processo nessa instância;
      obtem_valor(proc['dadosBasicos'], 'procEl', padrao=''), #dadosBasicos.procEl: campo identifica se o processo tramita em sistema eletrônico ou em papel. São valores possíveis - 1: Sistema Eletrônico - 2: Sistema Físico;
			obtem_valor(proc['dadosBasicos'], 'dscSistema', padrao='') #dadosBasicos.dscSistema: identifica qual o sistema eletrônico que o processo tramita. São valores possíveis: 1 – Pje, 2 – Projudi, 3 – SAJ, 4 – EPROC, 5 – Apolo, 6 – Themis, 7 – Libra, 8 – Outros;            			
  ]

def d_ass(proc):    
  id_processo = gera_id_processo (proc)
  assuntos = []
  if 'assunto' in proc['dadosBasicos'] and proc['dadosBasicos']['assunto'] != None:
    assuntos = proc['dadosBasicos']['assunto']
  for p in assuntos:
    yield [
           id_processo, #'id_processo'  TRT9_1125_G1_20393_00000014220195090325.json
           obtem_valor(p, 'codigoNacional', padrao=''), #dadosBasicos.assunto.codigoNacional: elemento destinado a que se informe o código de assunto existente na tabela nacional unificada decorrente da Resolução 46;
           obtem_valor(p, 'principal', padrao=''),  #dadosBasicos.assunto.principal: atributo do tipo boolean que informa se o assunto referido é o assunto principal do processo.
           obtem_valor(p, 'codigoAssunto', padrao=''), #dadosBasicos.assunto.assuntoLocal.codigoAssunto: atributo destinado a incluir a informação relativa ao código numérico utilizado localmente pelo tribunal.
			     obtem_valor(p, 'codigoPaiNacional', padrao=''), #dadosBasicos.assunto.assuntoLocal.codigoPaiNacional: atributo destinado à entrada do código de assunto nacional de que o assunto local é filho.
			     obtem_valor(p, 'descricao', padrao='') #dadosBasicos.assunto.assuntoLocal.descricao: atributo destinado à entrada da descrição textual do assunto local.
          ]


def d_mov(proc):
  movimentos = []
  id_processo = gera_id_processo (proc)
  if 'movimento' in proc and proc['movimento'] != None:
    movimentos = proc['movimento']
  for p in movimentos:
    if 'orgaoJulgador' in p and p['orgaoJulgador'] != None:
      codigoMunicipioIBGE = obtem_valor(p['orgaoJulgador'], 'codigoMunicipioIBGE', padrao='') #movimento.orgaoJulgador.dadosBasicos.codigoMunicipioIBGE: município-sede da unidade judiciária, conforme código de municípios definidos pelo IBGE. Usar código com sete dígitos. Fonte: www.ibge.gov.br;
      codigoOrgao = obtem_valor(p['orgaoJulgador'], 'codigoOrgao', padrao='') #movimento.orgaoJulgador.dadosBasicos.codigoOrgao: código da Unidade Judiciária constante no Módulo de Produtividade, Anexo II da Resolução 76.
      instancia = obtem_valor(p['orgaoJulgador'], 'instancia', padrao='') #movimento.orgaoJulgador.dadosBasicos.instancia: os tipos de instância podem ser: - ORIG: instância originária em que o processo teve início; - REV: instância de revisão direta de um processo originariamente proposto em outra instância; - ESP: instância de revisão especial de processo submetido ou não à revisão direta; - EXT: instância de revisão extraordinária - ADM: instância administrativa de análise;
    else:
      codigoMunicipioIBGE = ''
      codigoOrgao = ''
      instancia = ''

    movimentoNacional_codigoNacional = ''
    movimentoLocal_codigoMovimento = ''
    movimentoLocal_codigoPaiNacional = ''    
    if 'movimentoNacional' in p and p['movimentoNacional'] != None:
        movimentoNacional_codigoNacional = obtem_valor(p['movimentoNacional'],'codigoNacional', padrao='') #movimento.movimentoNacional.codigoNacional: atributo destinado à indicação do código do movimento previsto na tabela unificada de que trata a Resolução 46;      
    if 'movimentoLocal' in p and p['movimentoLocal'] != None:
        movimentoLocal_codigoMovimento = obtem_valor(p['movimentoLocal'], 'codigoMovimento', padrao='') #movimento.movimentoLocal.codigoMovimento: atributo destinado a incluir a informação relativa ao código numérico utilizado localmente pelo tribunal;
        movimentoLocal_codigoPaiNacional = obtem_valor(p['movimentoLocal'], 'codigoPaiNacional', padrao=''), #movimento.movimentoLocal.codigoPaiNacional: atributo destinado à entrada do código de movimento nacional de que o movimento local é filho;
    yield [
    id_processo, #
		obtem_valor(p, 'dataHora', padrao=''), ##movimento.dataHora: atributo destinado a indicar o momento em que foi realizada a movimentação.
		obtem_valor(p, 'nivelSigilo', padrao=''), #movimento.nivelSigilo: nível de sigilo a ser aplicado ao processo;
		obtem_valor(p, 'identificadorMovimento', padrao=''), #movimento.identificadorMovimento: atributo incluído para permitir a atribuição de um identificador específico para a movimentação realizada em um determinado processo judicial.
		obtem_valor(p, 'tipoResponsavelMovimento', padrao=''), #movimento.tipoResponsavelMovimento: identificação do responsável pelo movimento: Servidor=0; Magistrado=1;
		obtem_valor(p, 'idDocumentoVinculado', padrao=''), #movimento.idDocumentoVinculado: elemento destinado a permitir a vinculação de um ou mais documentos à movimentação;
    codigoMunicipioIBGE, #movimento.orgaoJulgador.dadosBasicos.codigoMunicipioIBGE: município-sede da unidade judiciária, conforme código de municípios definidos pelo IBGE. Usar código com sete dígitos. Fonte: www.ibge.gov.br;
		codigoOrgao, #movimento.orgaoJulgador.dadosBasicos.codigoOrgao: código da Unidade Judiciária constante no Módulo de Produtividade, Anexo II da Resolução 76. 
		instancia, #movimento.orgaoJulgador.dadosBasicos.instancia: os tipos de instância podem ser: - ORIG: instância originária em que o processo teve início; - REV: instância de revisão direta de um processo originariamente proposto em outra instância; - ESP: instância de revisão especial de processo submetido ou não à revisão direta; - EXT: instância de revisão extraordinária - ADM: instância administrativa de análise;
		obtem_valor(p, 'tipoDecisao', padrao=''), #movimento.tipoDecisao: atributo que permite a atribuição da decisão como sendomonocrática (proferida por um magistrado), ou colegiada;Valores possíveis são numéricos (0 ou 1):0 - decisão MONOCRATICA1 - decisão COLEGIADA;
		movimentoNacional_codigoNacional, #movimento.movimentoNacional.codigoNacional: atributo destinado à indicação do código do movimento previsto na tabela unificada de que trata a Resolução 46;
		movimentoLocal_codigoMovimento, #movimento.movimentoLocal.codigoMovimento: atributo destinado a incluir a informação relativa ao código numérico utilizado localmente pelo tribunal;
		movimentoLocal_codigoPaiNacional #movimento.movimentoLocal.codigoPaiNacional: atributo destinado à entrada do código de movimento nacional de que o movimento local é filho;
    ]
    
    
def d_compl(proc):
  complementos = []
  if 'movimento' in proc and proc['movimento'] != None and 'complementoNacional' in proc['movimento'] and proc['movimento']['complementoNacional'] != None:
    complementos = proc['movimento']['complementoNacional']
  for p in complementos:
      identificadorMovimento = str(p['identificadorMovimento'])
      yield [
      identificadorMovimento, #identificado movimento / chave
			obtem_valor(p,'codComplemento', padrao=''),#'complementoNacional_codComplemento', #	movimento.complementoNacional.codComplemento: atributo destinado à entrada do código docomplemento do movimentonacional ou do movimento local;
			obtem_valor(p,'descricaoComplemento', padrao=''), #	movimento.complementoNacional.descricaoComplemento: atributo destinado à entrada da descrição textualdocomplemento do movimento nacional ou do movimento local;
			obtem_valor(p,'codComplementoTabelado', padrao='') #movimento.complementoNacional.codComplementoTabelado: atributo destinado à entrada do código do complemento tabelado do movimento nacional ou do movimento local;
			]


## Procedimento para obter todos os arquivos da base disponibilizada e gerar CSVs

São 4 CSVS gerados: processos_1.csv, assuntos_1.csv, movimentos_1.csv e complementos_1.csv

Pode demorar um pouco (algo em torno de 15 minutos).

In [11]:

client = storage.Client(project=project)
bucket = storage.Bucket(client, name=BUCKET_NAME, user_project=project)
prefix = "base/"
all_blobs = list(client.list_blobs(bucket,prefix=prefix ))

blobs = [y for y in all_blobs[1:] if y.name.strip().endswith('.json')]

particao="_1"
csv_processos = f"processos{particao}.csv"
csv_assuntos = f"assuntos{particao}.csv"
csv_movimentos = f"movimentos{particao}.csv"
csv_complementos = f"complementos{particao}.csv"

with open(csv_processos, 'w', newline='') as f_processos, open(csv_assuntos,'w', newline='') as f_assuntos, open(csv_movimentos,'w', newline='') as f_movimentos, open(csv_complementos,'w', newline='') as f_complementos:
  w_processos = csv.writer(f_processos, quoting=csv.QUOTE_ALL)
  w_assuntos = csv.writer(f_assuntos)
  w_movimentos = csv.writer(f_movimentos)
  w_complementos = csv.writer(f_complementos)
  #cabeçalhos
  w_processos.writerows([campos['processos']])
  w_assuntos.writerows([campos['assuntos_processo']])
  w_movimentos.writerows([campos['movimentos_processo']])
  w_complementos.writerows([campos['complementos_movimento']])
  
  erros = []
  total_arquivos = 0
  for blob in blobs:
    try:
      blob_name = blob.name.lstrip(prefix).replace(".json","")
      blob_content = blob.download_as_string()
      proc_json = json.loads(blob_content)
      numero_item = -1
      for p in proc_json:
        total_arquivos = total_arquivos + 1
        numero_item = numero_item + 1
        w_processos.writerows(d_proc(p))
        w_assuntos.writerows(d_ass(p))
        w_movimentos.writerows(d_mov(p))
        w_complementos.writerows(d_compl(p))
        #Imprime datahora a cada 50.000 arquivos
        if total_arquivos % 50000 == 0:
          datahora = datetime.now().strftime("%d/%m/%y %H:%M")
          print(f"{datahora};{total_arquivos}")
    except Exception as e: 
      erro = {
          'arquivo': blob_name,
          'numero_item': numero_item,
          'erro': e,
          'traceback':traceback.format_exc()
      }
      erros.append(erro)
      logger.warning(erro)
    
      
  
    #dadosBasicos
  



21/10/20 15:46;50000
21/10/20 15:46;100000
21/10/20 15:46;150000
21/10/20 15:47;200000
21/10/20 15:47;250000
21/10/20 15:47;300000
21/10/20 15:48;350000
21/10/20 15:49;400000


KeyboardInterrupt: ignored

## Verifica se houve algum erro de importação:

In [14]:
print(erros)

[]


## Com os CSVs gerados no ambiente do transiente do Google Colab, faz upload para o Google Storage na pasta "base_csv"

In [ ]:
arquivos = ['processos_1.csv','assuntos_1.csv','movimentos_1.csv','complementos_1.csv']
for arquivo in arquivos:
  print(arquivo)
  blob = bucket.blob(f"base_csv/{arquivo}")
  blob.upload_from_filename(arquivo)


processos_1.csv
assuntos_1.csv
movimentos_1.csv
complementos_1.csv


## Upload dados via API python para BIgQuery:

Os CSVs gerados podem ser utilizados diretamente ou importados para qualquer tecnologia de banco de dados, como o Google BigQuery no Gloogle Cloud Platform

In [ ]:
from google.cloud import bigquery
file_path = "processos_1.csv"
dataset = "processos1"
#table = "your_table_name"
table = file_path.replace(".csv","")
# Construct a BigQuery client object.
client = bigquery.Client(project=project)

# TODO(developer): Set table_id to the ID of the table to create.
table_id = f"{project}.{dataset}.{table}"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
)

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()  # Waits for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)


Loaded 6 rows and 21 columns to tensile-virtue-291314.processos1.processos_1
